# Summary

Start tying openai and youtube functionality together to manage the punctuation process.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from youtube_transcript_api import YouTubeTranscriptApi, NoTranscriptFound

from jabberwocky.config import C
from jabberwocky.openai_utils import query_gpt3
from jabberwocky.utils import load_prompt
from jabberwocky.youtube import realign_punctuated_text, get_transcripts, \
    text_segment, video_id
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [195]:
class UnpunctuatedTranscript:
    
    def __init__(self, df_gen, **kwargs):
        self.df_gen = df_gen
        self.df_punct = self.df_gen.copy()
        self.df_punct['text'] = np.nan
        # TODO: adjust so prompt template saved separately from prompt_kwargs.
        # TODO: adjust load_prompt to have option to skip .format call 
        # if no text passed in.
        self.prompt_kwargs = load_prompt('punctuate')
        self.prompt_fmt = self.prompt_kwargs.pop('prompt')
        self.prompt_kwargs.update(kwargs)
        
    @property
    def df(self):
        # TODO: maybe should return self.df_punct, possibly after punct query? 
        # Unsure.
        return self.df_gen
    
    def _punctuate_chunk(self, df_chunk, **kwargs) -> str:
        text = ' '.join(df_chunk.text)
        prompt = self.prompt_fmt.format(text).rstrip()
        # n_tokens is roughly 1.33 * n_words. Add buffer for punctuation.
        prompt_kwargs = {**self.prompt_kwargs,
                         **kwargs, 
                         'max_tokens': int(len(text.split()) * 2)}
        _, text_punct = query_gpt3(prompt, **prompt_kwargs)
        return text_punct
    
#     def punctuated_chunk(self, start_idx, end_idx, align_kwargs=None, 
#                         **query_kwargs):
#         df_chunk = self.df_gen.loc[start_idx:end_idx]
#         for idx in self.na_index_chunks(self.df_punct.loc[start_idx:end_idx]):
#             print(idx)
#         return
        
        
#         text_punct = self._punctuate_chunk(df_chunk, **query_kwargs)
#         df_chunk_punct = realign_punctuated_text(df_chunk, text_punct, 
#                                                  **ifnone(align_kwargs, {}))
#         # Must use 'values' attribute because realignment func resets index.
#         self.df_punct.loc[start_idx:end_idx, 'text'] = df_chunk_punct.text.values
#         return self.df_punct.loc[start_idx:end_idx, :]
    
    # This version only punctuates rows of the relevant chunk that haven't 
    # been previously punctuated. While this is faster and cheaper, I've seen
    # some hints that the punctuation task may work better when we pass it
    # long-ish chunks of text and not little partial snippets. So it might
    # actually be better not to do this? I was going to say it might do better
    # if given full sentences rather than fragments, but I guess we can't 
    # easily extract those without doing the actual punctuation step.
    def punctuated_chunk(self, start_idx, end_idx, punctuate, 
                         align_kwargs=None, **query_kwargs):
        if not punctuate:
            return self.df_gen.loc[start_idx:end_idx, :]
        for idx in self.na_index_chunks(self.df_punct.loc[start_idx:end_idx]):
            df_chunk = self.df_gen.loc[idx]
            text_punct = self._punctuate_chunk(df_chunk, **query_kwargs)
            df_chunk_punct = realign_punctuated_text(
                df_chunk, text_punct, **ifnone(align_kwargs, {})
            )
            # Use 'values' attribute because realignment func resets index.
            self.df_punct.loc[idx, 'text'] = df_chunk_punct.text.values
        return self.df_punct.loc[start_idx:end_idx, :]
    
    @staticmethod
    def na_index_chunks(chunk) -> list:
        """Given a chunk of a df that may contain null text rows, return a 
        list of lists where each nested list contains the indices of a 
        contiguous chunk of null rows.
        """
        nans = chunk[chunk.text.isnull()]
        if nans.empty: return []
        last_idx = nans.index[-1]
        res = []
        curr_chunk = []
        prev = None
        for idx in nans.index:
            if prev is None or (idx == prev + 1): 
                curr_chunk.append(idx)
            else:
                res.append(curr_chunk)
                curr_chunk = [idx]
            if idx == last_idx:
                res.append(curr_chunk)
            prev = idx
        return res

In [196]:
class PunctuatedTranscript:
    
    def __init__(self, df_gen, df_punct, **kwargs):
        """
        kwargs:
            Just for compatibility with UnpunctuatedTranscript, which needs
            these to specify args like 'rstrip' when loading a prompt.
        """
        self.df_gen = df_gen
        self.df_punct = df_punct
        
    @property
    def df(self):
        return self.df_punct
    
    def punctuated_chunk(self, start_idx, end_idx, **kwargs):
        return self.df_punct.loc[start_idx:end_idx, :]

In [197]:
@auto_repr
class Transcript:
    
    def __init__(self, url, **kwargs):
        self.url = url
        self.id = video_id(url)
        self._transcript = self._fetch_transcripts(url, **kwargs)
        self.is_generated = isinstance(self._transcript,
                                       UnpunctuatedTranscript)
        self.start_time, self.end_time = self.df.start.ends(1)
            
    def _time_range(self, start, end, full_sentences=True):
        assert end > start, 'End time must be later than start time.'
        assert start >= 0 and end >= 0, 'Times must be non-negative.'

        df = self.df
        if start < self.start_time:
            start_idx = 0
        else:
            start_idx = df.loc[df.start <= start].index[-1]

        if end > df.start.iloc[-1]:
            end_idx = df.tail(1).index[0]
        else:
            end_idx = df.loc[df.start >= end].index[0]
        return df.iloc[start_idx:end_idx+1]
    
    def time_range(self, start, end, punctuate=False, full_sentences=True,
                   align_kwargs=None, **query_kwargs):
        chunk = self._time_range(start, end, full_sentences=full_sentences)
        return self._transcript.punctuated_chunk(*chunk.ends(1).index, 
                                                 punctuate=punctuate,
                                                 align_kwargs=align_kwargs,
                                                 **query_kwargs)
    
    @property
    def df(self):
        return self._transcript.df
    
    def _fetch_transcripts(self, url, **kwargs):
        """Wrapper to fetch youtube transcripts and create the appropriate
        transcript object depending on whether a manually generated (i.e.
        punctuated) transcript was retrieved.
        
        Parameters
        ----------
        url: str
        verbose: bool
        """
        df_gen, df_man, _ = self.get_transcripts(
            url, verbose=kwargs.get('verbose', True)
        )
        if df_man is None:
            return UnpunctuatedTranscript(df_gen, **kwargs)
        else:
            return PunctuatedTranscript(df_gen, df_man, **kwargs)
        
    @staticmethod
    def get_transcripts(url, verbose=True):
        """Fetch one or more transcripts for a youtube video given its URL.

        Parameters
        ----------
        url: str
            Don't include any channel-related suffix. E.G. use
            https://www.youtube.com/watch?v=OZbCRN3C_Hs, not
            https://www.youtube.com/watch?v=OZbCRN3C_Hs&ab_channel=BBC.
        verbose: bool
            Warn

        Returns
        -------
        DotDict: Contains keys 'id' (maps to video ID str), 'generated',
        and 'manual' (the latter two lap to pandas dfs or None if no 
        transcript was found). Manual transcripts are human-created.
        Generated transcripts are a bit lower quality and tend to lack
        punctuation.
        """
        langs = ['en', 'en-GB']
        id_ = video_id(url)
        res = {'generated': None, 'manual': None}
        trans_list = YouTubeTranscriptApi.list_transcripts(id_)
        res['generated'] = trans_list.find_generated_transcript(langs)
        try:
            res['manual'] = trans_list.find_manually_created_transcript(langs)
        except NoTranscriptFound:
            if verbose: warnings.warn('No manual transcript found.')
        if verbose:
            non_eng = [k for k, v in res.items()
                       if v and ('United Kingdom' in v.language)]
            if non_eng:
                warnings.warn(
                    f'{non_eng} {"has" if len(non_eng) == 1 else "have"} '
                    'language en-GB, not en.'
                )
        return Args(**{k: pd.DataFrame(v.fetch()) if v else v 
                       for k, v in res.items()},
                    id=id_)
    
    def __str__(self):
        return f'{type(self).__name__}(url={self.url}, '\
               f'is_generated={self.is_generated})'

In [189]:
gen_url = 'https://www.youtube.com/watch?v=AtTsn1Ia4JY&ab_channel=LukeThomas'
man_url = 'https://www.youtube.com/watch?v=NNnIGh9g6fA'

In [190]:
trans_gen = Transcript(gen_url)
trans_gen

/Users/hmamin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:87: UserWarning: No manual transcript found.


You should probably adjust max_tokens based on the length of the input. Bumping up to engine 2 or 3 might help a little, but engine 1 is serviceable (probably best to avoid 0 though). You should probably try training a huggingface model to add punctuation instead of using gpt3 credits though.
-------------------------------------------------------------------------------



Transcript(url='https://www.youtube.com/watch?v=AtTsn1Ia4JY&ab_channel=LukeThomas')

In [191]:
trans_man = Transcript(man_url)
trans_man

Transcript(url='https://www.youtube.com/watch?v=NNnIGh9g6fA')

In [192]:
trans_man.time_range(7, 16)

,text,start,duration
1,Stanford University.,4.94,0.92
2,"This is BIO 150, isn't it?",9.36,3.37
3,"OK, just wanted to make sure.",12.73,2.74
4,So we start off with a scenario.,15.47,3.42
5,"40-year-old guy--\nquiet, suburban life.",18.89,4.06


In [193]:
trans_gen._time_range(60, 70)

,text,start,duration
35,stuff but there is,58.879,2.801
36,a little bit more to the story what was,60.160,3.440
37,dan hooker trying what was his,61.680,3.839
38,game plan here and more to the point,63.600,4.160
39,there are some finer details to that,65.519,3.121
40,finish that,67.760,3.520
41,really deserve to get mentioned and i,68.640,4.080
42,don't know exactly how far,71.280,2.640


In [194]:
trans_gen.time_range(60, 70)

,text,start,duration
35,stuff but there is,58.879,2.801
36,a little bit more to the story what was,60.160,3.440
37,dan hooker trying what was his,61.680,3.839
38,game plan here and more to the point,63.600,4.160
39,there are some finer details to that,65.519,3.121
40,finish that,67.760,3.520
41,really deserve to get mentioned and i,68.640,4.080
42,don't know exactly how far,71.280,2.640


In [177]:
res = trans_gen.time_range(60, 70, True)

[35, 36, 37, 38, 39, 40, 41, 42]


/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 37: dan hooker trying what was his
  'Max score < 80. Your rows may have gotten misaligned '


In [178]:
res

,text,start,duration
35,stuff but there is,58.879,2.801
36,a little bit more to the story. What was,60.160,3.440
37,Dan Hooker,61.680,3.839
38,"trying to do here? And more to the point,",63.600,4.160
39,there are some finer details to that,65.519,3.121
40,finish that,67.760,3.520
41,really deserve to get mentioned. And I,68.640,4.080
42,don't know exactly how far,71.280,2.640


In [179]:
res = trans_gen.time_range(50, 80, True)

[28, 29, 30, 31, 32, 33, 34]


/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 30: at like what happened to be like oh
  'Max score < 80. Your rows may have gotten misaligned '
/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 31: michael chandler him with a big ass
  'Max score < 80. Your rows may have gotten misaligned '
/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 32: punch
  'Max score < 80. Your rows may have gotten misaligned '


[43, 44, 45, 46, 47]


/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 45: he was early in bellator in his you know
  'Max score < 80. Your rows may have gotten misaligned '


In [180]:
res

,text,start,duration
28,"and um as always, the devil is in the",49.280,4.240
29,details it doesn't take a genius to look,51.840,2.960
30,at like what,53.520,2.480
31,"happened to be like, Michael Chandler, him with a",54.800,1.680
32,"big ass punch,",56.000,1.760
33,"you know, and kind of faded low and then",56.480,2.399
34,went high and then all that kind of,57.760,2.400
35,stuff but there is,58.879,2.801
36,a little bit more to the story. What was,60.160,3.440
37,Dan Hooker,61.680,3.839


In [181]:
trans_gen._transcript.df_punct.loc[25:50]

,text,start,duration
25,NaN,44.719,3.520
26,NaN,46.800,2.480
27,NaN,48.239,3.601
28,"and um as always, the devil is in the",49.280,4.240
29,details it doesn't take a genius to look,51.840,2.960
30,at like what,53.520,2.480
31,"happened to be like, Michael Chandler, him with a",54.800,1.680
32,"big ass punch,",56.000,1.760
33,"you know, and kind of faded low and then",56.480,2.399
34,went high and then all that kind of,57.760,2.400


In [184]:
# Much faster this time because it's been pre-punctuated.
res = trans_gen.time_range(50, 80, True)

In [185]:
res

,text,start,duration
28,"and um as always, the devil is in the",49.280,4.240
29,details it doesn't take a genius to look,51.840,2.960
30,at like what,53.520,2.480
31,"happened to be like, Michael Chandler, him with a",54.800,1.680
32,"big ass punch,",56.000,1.760
33,"you know, and kind of faded low and then",56.480,2.399
34,went high and then all that kind of,57.760,2.400
35,stuff but there is,58.879,2.801
36,a little bit more to the story. What was,60.160,3.440
37,Dan Hooker,61.680,3.839


In [170]:
def na_index_chunks(chunk):
    nans = chunk[chunk.text.isnull()]
    if nans.empty: return []
    last_idx = nans.index[-1]
    res = []
    curr_chunk = []
    prev = None
    for idx in nans.index:
        if prev is None or (idx == prev + 1): 
            curr_chunk.append(idx)
        else:
            res.append(curr_chunk)
            curr_chunk = [idx]
        if idx == last_idx:
            res.append(curr_chunk)
        prev = idx
    return res

In [117]:
na_index_chunks(tmp.head(3))

[[32, 33, 34]]

In [116]:
eprint([tmp.loc[row] for row in na_index_chunks(tmp)])

 0:    text  start  duration
32  NaN  56.00     1.760
33  NaN  56.48     2.399
34  NaN  57.76     2.400
 1:    text  start  duration
46  NaN  78.88     4.720
47  NaN  80.72     4.079
48  NaN  83.60     1.920


In [249]:
res = trans_gen.time_range(10, 30)

/Users/hmamin/jabberwocky/lib/jabberwocky/youtube.py:164: UserWarning: Max score < 80. Your rows may have gotten misaligned at row 11: story
  'Max score < 80. Your rows may have gotten misaligned '


[PosixPath('data/prompts/simplify_ml'),
 PosixPath('data/prompts/shortest'),
 PosixPath('data/prompts/how_to'),
 PosixPath('data/prompts/README.md'),
 PosixPath('data/prompts/punctuate'),
 PosixPath('data/prompts/tldr'),
 PosixPath('data/prompts/eli'),
 PosixPath('data/prompts/short_dates')]

In [296]:
from functools import partialmethod

In [317]:
@auto_repr
class PromptManager:
    """Simple class that stores all the prompt templates and default kwargs
    so we don't need to load them repeatedly. Use this as an interface for
    performing tasks on a video Transcript object.
    """
    
    def __init__(self):
        self.prompts = self._load_templates()
        # TODO: prob rm. Kind of liked the idea of making each task its own
        # method (e.g. manager.tldr('...') instead of 
        # manager.query('tldr', '...')). However, using partial obscures the
        # kwargs which is annoying. For this to be worth it, I think I'd have
        # to make something that used some magic to load the default kwargs, 
        # inject them in as default kwargs, and make them viewable with shift
        # tab. The benefits probably don't justify that level of effort and
        # hackiness. I think a debug option is closer enough, but I'll sleep
        # on it before deleting this.
#         for name in self.prompts:
#             meth = method(self.query, task=name)
#             setattr(self, name, update_wrapper(meth, self.query))
    
    def _load_templates(self):
        name2kwargs = {}
        for path in Path('data/prompts').iterdir():
            if not path.is_dir(): continue
            name2kwargs[path.stem] = load_prompt(path.stem)
        return name2kwargs
    
    def query(self, task, text, debug=False, **kwargs):
        kwargs = {**self.prompts[task], **kwargs}
        prompt = kwargs.pop('prompt').format(text)
        if debug:
            print('prompt:\n' + prompt)
            print('kwargs:\n', kwargs)
            return
        return query_gpt3(prompt, **kwargs)
    
    def kwargs(self, task):
        return select(self.prompts[task], drop=['prompt'])
    
    def prompt(self, task, text='', print_=False):
        template = self.prompts[task]['prompt']
        res = template.format(text) if text else template
        if print_:
            print(res)
        else:
            return res
    
    def __str__(self):
        return f'{type(self).__name__}(prompts=[{", ".join(self.prompts)}])'

In [307]:
manager = PromptManager()
print(manager)

simplify_ml: This uses the expensive davinci model and doesn't work so well without it. Temperature is set to 0.3 but this hasn't been extensively tuned.
-------------------------------------------------------------------------------

shortest: This prompt takes no input.
-------------------------------------------------------------------------------

how_to: Should be a single line starting with the words "How to" and ending in a colon. You may need a stronger engine for good results.
-------------------------------------------------------------------------------

punctuate: You should probably adjust max_tokens based on the length of the input. Bumping up to engine 2 or 3 might help a little, but engine 1 is serviceable (probably best to avoid 0 though). You should probably try training a huggingface model to add punctuation instead of using gpt3 credits though.
-------------------------------------------------------------------------------

tldr: This sets max tokens to 64. You may 

In [308]:
manager.prompt('tldr')

'{}\n\ntl;dr:'

In [310]:
manager.prompt('tldr', 'abcd')

'abcd\n\ntl;dr:'

In [311]:
manager.prompt('tldr', 'abcd', print_=True)

abcd

tl;dr:


In [312]:
manager.kwargs('tldr')

{'engine_i': 2, 'max_tokens': 64, 'temperature': 0.3}

In [313]:
vars(manager).keys()

dict_keys(['prompts', 'simplify_ml', 'shortest', 'how_to', 'punctuate', 'tldr', 'eli', 'short_dates'])

In [316]:
manager.query('eli', 'Attention enables the network to learn dynamic linear layers, essentially encoding the input sequence as a directed, weighted graph which we represent via a learned adjacency matrix.', debug=True, stream=True, engine_i=1)

prompt:
Attention enables the network to learn dynamic linear layers, essentially encoding the input sequence as a directed, weighted graph which we represent via a learned adjacency matrix. 

I rephrased this for my daughter, in plain language a second grader can understand:
kwargs:
 {'engine_i': 1, 'temperature': 0.3, 'stop': 'I rephrased this for my daughter, in plain language', 'stream': True}


In [273]:
manager.eli(text='Batch normalization decouples layer outputs from one another '
             'and reduces internal covariate shift. It leads to faster '
             'optimization by enabling the neural network to undergo stable '
             'training even with large learning rates.')

('Batch normalization decouples layer outputs from one another and reduces internal covariate shift. It leads to faster optimization by enabling the neural network to undergo stable training even with large learning rates. \n\nI rephrased this for my daughter, in plain language a second grader can understand:',
 'Batch normalization is a way to make the math easier when you are trying to learn how to do something.\n\nThe paper is very readable and I highly recommend it.\n\nThe paper is also available in the arXiv')

In [284]:
manager.query('eli', 'Batch normalization decouples layer outputs from one another '
             'and reduces internal covariate shift. It leads to faster '
             'optimization by enabling the neural network to undergo stable '
             'training even with large learning rates.', engine_i=3)

('Batch normalization decouples layer outputs from one another and reduces internal covariate shift. It leads to faster optimization by enabling the neural network to undergo stable training even with large learning rates. \n\nI rephrased this for my daughter, in plain language a second grader can understand:',
 'Batch normalization is a way to make the learning process faster.\n\nBatch normalization is a technique that allows you to use larger learning rates.\n\nBatch normalization makes the neural network more stable.\n\nB')